# Equilibrium of a Solid Biomass Fuel with Air

In this notebook we study the thermochemical equilibrium of a simplified
solid biomass fuel (represented by its elemental composition) reacting with
air. The goal is to understand how Cantera can be used to:

- specify a fuel in terms of elemental moles (C, H, O, N, moisture),
- determine the amount of oxidizing air required,
- build a multiphase `Mixture` with a gas phase (syngas) and a solid carbon
  phase (char/graphite),
- perform an equilibrium calculation at fixed temperature and pressure,
- extract the moles of the main products (H2, H2O, CO, CO2, CH4, N2, O2, C(gr)).

The numerical values here are purely illustrative. After running the cell,
experiment with the fuel composition (e.g. more/less moisture, different H/C
ratio) and the temperature to see how the equilibrium gas composition changes.


In [ ]:
# Import necessary libraries
import cantera as ct
import numpy as np

# --- Parameters ---
T = 500 + 273.15  # Temperature in Kelvin
P = 101325  # Pressure in Pascals

# Initial moles of elements in the biomass composition
C = 1.0
H = 1.52
O = 0.67
N = 0.01
H2O = 0.3

# Oxidizing air requirements
O2 = C + H / 4 - O / 2
N2 = 79 / 21 * O2

# Initial total number of moles (sum of reactants)
moli_IN = H2O + H + O + N + O2 + N2
Ash = 0  # Ash content, optional

# Initialize the gas-phase (syn) and solid-phase (carbon) models
syn = ct.Solution("gri30.yaml")
carbon = ct.Solution("graphite.yaml")

# Set initial gas-phase composition and operating conditions
syn.TPX = (
    T,
    P,
    {
        "H2": 0,
        "H2O": H2O,
        "CO": 0,
        "CO2": 0,
        "CH4": 0,
        "N2": N2,
        "O2": O2,
        "C": 0,
        "H": H,
        "O": O,
        "N": N,
    },
)

# Create a mixture with the gas and solid phases
mix = ct.Mixture([(syn, moli_IN), (carbon, C)])

# Set state of the mixture
mix.T = T
mix.P = P

# Equilibrium calculation at constant temperature and pressure
mix.equilibrate("TP")

# --- Results extraction ---
# Gas and solid phase mole fractions
CR1 = syn.X
CR2 = carbon.X

# Get species indices for relevant species in the gas phase
ih2 = syn.species_index("H2")
ih2o = syn.species_index("H2O")
ico = syn.species_index("CO")
ico2 = syn.species_index("CO2")
ich4 = syn.species_index("CH4")
in2 = syn.species_index("N2")
io2 = syn.species_index("O2")

# For solid carbon in the carbon phase
icgr = carbon.species_index("C(gr)")

# Phase mole numbers
Moligas = mix.phase_moles(0)  # Number of moles in the gas phase
Molicarbon = mix.phase_moles(1)  # Number of moles in the solid phase

# Mean molecular weights
Pmgas = syn.mean_molecular_weight
Pmcarbon = carbon.mean_molecular_weight

# Mass of each phase
Qmgas = Moligas * Pmgas
Qmcarbon = Molicarbon * Pmcarbon
Qmtot = Qmgas + Qmcarbon  # Total mass

# Moles of products in each phase
Qgas = Qmgas / Pmgas
Qcarbon = Qmcarbon / Pmcarbon

# Retrieve species names
species_names = syn.species_names

# Moles of each gaseous product
QH2 = CR1[ih2] * Qgas
QH2O = CR1[ih2o] * Qgas
QCO = CR1[ico] * Qgas
QCO2 = CR1[ico2] * Qgas
QCH4 = CR1[ich4] * Qgas
QN2 = CR1[in2] * Qgas
QO2 = CR1[io2] * Qgas
QCgr = CR2[icgr] * Qcarbon  # Solid carbon (graphite)

# Filter out negligible mole fractions
filtered_CR1 = np.where(CR1 < 1e-4, 0, CR1)

# Display results
print("Equilibrium Composition of Selected Products (Moles):")
print(f"H2: {QH2:.4f}")
print(f"H2O: {QH2O:.4f}")
print(f"CO: {QCO:.4f}")
print(f"CO2: {QCO2:.4f}")
print(f"CH4: {QCH4:.4f}")
print(f"N2: {QN2:.4f}")
print(f"O2: {QO2:.4f}")
print(f"C(gr): {QCgr:.4f}")

Equilibrium Composition of Selected Products (Moles):
H2: 0.0000
H2O: 1.0600
CO: 0.0000
CO2: 1.0000
CH4: 0.0000
N2: 3.9362
O2: 0.0000
C(gr): 0.0000
